In [ ]:
import numpy as np
import pandas as pd

In [ ]:
movies=pd.read_csv('../input/tmdb-movie-metadata/tmdb_5000_movies.csv')
credits=pd.read_csv('../input/tmdb-movie-metadata/tmdb_5000_credits.csv')

In [ ]:
movies.head()

In [ ]:
credits.head()

In [ ]:
movies=movies.merge(credits, on='title')

In [ ]:
movies.head()

In [ ]:
# UNWANTED COLUMNS
# budget
# homepage
# id
# original_language
# original_title
# popularity
# production_comapny
# production_countries
# release-date(not sure)

In [ ]:
movies = movies[['movie_id','title','overview','genres','keywords','cast','crew']]

In [ ]:
movies.head()

In [ ]:
movies.isnull().sum()

In [ ]:
movies.dropna(inplace=True)

In [ ]:
movies.duplicated().sum()

In [ ]:
movies.iloc[0].genres

In [ ]:
import ast
def convert(object):
    list=[]
    for i in ast.literal_eval(object):
        list.append(i['name'])
    return list

In [ ]:
movies['genres']=movies['genres'].apply(convert)

In [ ]:
movies.head()

In [ ]:
movies['keywords']=movies['keywords'].apply(convert)

In [ ]:
movies.head()

In [ ]:
import ast
def convert3(obj):
    list=[]
    count=0
    for i in ast.literal_eval(obj):
        if count != 3:
            list.append(i['name'])
            count+=1
        else:
            break
    return list

In [ ]:
movies['cast'] = movies['cast'].apply(convert3)

In [ ]:
movies.head()

In [ ]:
def director(text):
    L = []
    for i in ast.literal_eval(text):
        if i['job'] == 'Director':
            L.append(i['name'])
    return L 

In [ ]:
movies['crew'] = movies['crew'].apply(director)

In [ ]:
movies.head()

In [ ]:
movies['overview']=movies['overview'].apply(lambda x:x.split())

In [ ]:
movies.head()

In [ ]:
movies['genres']=movies['genres'].apply(lambda x:[i.replace(" ","") for i in x])
movies['keywords']=movies['keywords'].apply(lambda x:[i.replace(" ","") for i in x])
movies['crew']=movies['crew'].apply(lambda x:[i.replace(" ","") for i in x])
movies['cast']=movies['cast'].apply(lambda x:[i.replace(" ","") for i in x])

In [ ]:
movies.head()

In [ ]:
movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']

In [ ]:
movies.head()

In [ ]:
movies=movies[['movie_id','title','tags']]

In [ ]:
movies.head()

In [ ]:
movies['tags']=movies['tags'].apply(lambda x:" ".join(x))

In [ ]:
movies.head()

In [ ]:
movies['tags']=movies['tags'].apply(lambda x:x.lower())

In [ ]:
movies.head()

# Vectorization 
### Bag of Words Technique

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
v = CountVectorizer(max_features =5000, stop_words='english')

In [ ]:
vectors=v.fit_transform(movies['tags']).toarray()

In [ ]:
vectors

In [ ]:
vectors[0]

In [ ]:
v.get_feature_names()

# Steming

In [ ]:
import nltk
from nltk.stem.porter import PorterStemmer

In [ ]:
ps=PorterStemmer()

In [ ]:
def stem(text):
    li=[]
    for i in text.split():
        li.append(ps.stem(i))
    string=" ".join(li)
    return string
    

In [ ]:
movies['tags']=movies['tags'].apply(stem)

# Measuring Cosine Distance b/w vectors

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
similarity =cosine_similarity(vectors)

In [ ]:
similarity[0]

In [ ]:
def recommend(movie):
    movie_index=movies[movies['title']==movie].index[0]
    distances=similarity[movie_index]
    movies_list=sorted(list(enumerate(distances)), reverse=True ,key=lambda x:x[1])[1:6]
    for i in movies_list:
         print(movies.iloc[i[0]].title)
       
    

In [ ]:
recommend('Avatar')

In [ ]:
movies.head()

# Exporting to Create GUI


In [ ]:
import pickle

In [ ]:
pickle.dump(movies.to_dict(),open('movies.pkl','wb'))

In [ ]:
pickle.dump(similarity,open('similarity.pkl','wb'))

## Creating A Streamlit App Using The Movies DataFrame and Cosine Similarity

### Create a python file and copy the below code and run to see the Web APP
#### and please use your own api key to get the posters i am not hiding my api key

In [ ]:
!pip install streamlit
import streamlit as st
import requests
import pickle

@st.cache
def get_posters(movie_id):
    data=requests.get("https://api.themoviedb.org/3/movie/{}?api_key=8265bd1679663a7ea12ac168da84d2e8&language=en-US".format(movie_id))
    response=data.json()
    poster_path = response['poster_path']
    
    return "https://image.tmdb.org/t/p/w500/" + poster_path

def recommend(movie):
    movie_index = movies[movies['title']==movie].index[0]
    distances=similarity[movie_index]
    movies_list=sorted(list(enumerate(distances)), reverse=True ,key=lambda x:x[1])[1:6]
    recommended=[]
    movie_posters=[]
    for i in movies_list:
        movie_id=movies.iloc[i[0]].movie_id
        recommended.append((movies.iloc[i[0]].title))
        movie_posters.append(get_posters(movie_id))
    return recommended,movie_posters

st.title('Movies Recommendation System')
movies=pickle.load(open('./movies.pkl','rb'))
movies=pd.DataFrame(movies)
selected = st.selectbox('Enter or Select Movie Name',
movies['title'].values)

similarity=pickle.load(open('./similarity.pkl','rb'))



if st.button('Recommend'):
    names,posters= recommend(selected)
    col1, col2, col3, col4, col5 = st.columns(5)
    with col1:
        st.text(names[0])
        st.image(posters[0])
    with col2:
        st.text(names[1])
        st.image(posters[1])

    with col3:
        st.text(names[2])
        st.image(posters[2])
    with col4:
        st.text(names[3])
        st.image(posters[3])
    with col5:
        st.text(names[4])
        st.image(posters[4])
